In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import etfs as et

In [ ]:
from etfs.security.io import read_yahoo_csv
from etfs.visualization.trends import plot_trend, plot_candlestick, plot_ewm

## Read all data at once

In [ ]:
def read_data(securities = ['VO', 'VOO', 'BNDX'], start = '2017-01-01', end = '2018-12-31'):
    out_df = pd.DataFrame(index=pd.DatetimeIndex(start=start, end=end, freq='D'))
    for security in securities:
        filepath = '../data/{0}.csv'.format(security)
        df = read_yahoo_csv(path=filepath)
        for column in df.columns:
            out_df[(column, security)] = df[column]
    
    out_df.dropna(how='all', axis=0, inplace=True)
    out_df.columns = pd.MultiIndex.from_tuples(out_df.columns)
    out_df = out_df.sortlevel(axis=1, ascending=True)
    
    return out_df

In [ ]:
df_in = pd.read_csv("../data/portfolio_rh.csv", parse_dates=[0])

In [ ]:
df_in.head()

In [ ]:
securities = [security for security in df_in.Ticker.unique() if security is not np.nan]

In [ ]:
df_in.Date.min()

In [ ]:
df_in.Date.max()

In [ ]:
df = read_data(securities=securities, start=df_in.Date.min(), end=df_in.Date.max())

In [ ]:
for security in securities:
    plt.plot(df['Close'].index, df['Close'][security], label=security)
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
df.head()

In [ ]:
user = input("User:")

In [ ]:
r.login()

In [ ]:
import robin_stocks as r

In [ ]:
import os
import robin_stocks.helper as helper
import robin_stocks.urls as urls
import robin_stocks.stocks as stocks
import robin_stocks.profiles as profiles

In [ ]:
import getpass


In [ ]:
getpass.getpass("Enter your password: ")

In [ ]:
# access password file
f = open("../../../.etfs.txt","r")
lines = f.readlines()
username = lines[0].strip().split('\n', 0)
password = lines[1].strip().split('\n', 0)
f.close()

In [ ]:
r.login(username, password)

In [ ]:
r.build_holdings()

In [ ]:
r.account.get_all_positions()

In [ ]:
r.download_all_documents()

In [ ]:
r.get_notifications()

In [ ]:
r.get_wire_transfers()

In [ ]:
r.get_bank_transfers()

In [ ]:
orders = r.get_all_orders()

In [ ]:
orders[0]

In [ ]:
len(orders)

https://github.com/jmfernandes/robin_stocks/blob/master/robin_stocks/stocks.py

In [ ]:
orders[0]['executions']

In [ ]:
r.get_instrument_by_url('https://api.robinhood.com/instruments/96d05e43-1b28-4a0c-9904-063b6f989cc0/')

In [ ]:
r.get_instrument_by_url(orders[0]['instrument'])

In [ ]:
r.get_latest_price('BIDU')

### Date

In [ ]:
order_time = pd.to_datetime(orders[0]['last_transaction_at'])

In [ ]:
order_time

### Transaction

In [ ]:
orders[0]['side']

### Ticker

In [ ]:
r.get_instrument_by_url(orders[0]['instrument'])['symbol']

### Price

In [ ]:
orders[0]['average_price']

### Quantity

In [ ]:
orders[0]['quantity']

### Build dataframe

In [ ]:
Date = []
Transaction = []
Ticker = []
Currency = []
Price = []
Quantity = []
OrderNo = []

In [ ]:
i = 0
for order in orders:
    print(i)
    if len(order['executions']):
        Date.append(pd.to_datetime(order['last_transaction_at']))
        Transaction.append(order['side'])
        Ticker.append(r.get_instrument_by_url(order['instrument'])['symbol'])
        Currency.append('USD')
        Price.append(order['average_price'])
        Quantity.append(order['quantity'])
        OrderNo.append(i)
    i = i + 1

### Add deposits

In [ ]:
transfers = r.get_bank_transfers()

In [ ]:
len(transfers)

In [ ]:
transfers[0]

In [ ]:
i = 0
for transfer in transfers:
    if transfer['cancel'] is None:
        Date.append(pd.to_datetime(transfer['created_at']))
        Transaction.append('deposit')
        Ticker.append(None)
        Currency.append('USD')
        Price.append(1.0)
        Quantity.append(transfer['amount'])
        OrderNo.append(i)
    i = i + 1    

### Include the free CHK stock

In [ ]:
Date.append(pd.to_datetime('2/1/18'))

In [ ]:
Transaction.append('buy')

In [ ]:
Ticker.append('CHK')

In [ ]:
Currency.append('USD')

In [ ]:
Price.append(0.0)

In [ ]:
Quantity.append(1.0)

### Dividends

In [ ]:
dividends = r.get_dividends()

In [ ]:
dividends[0]

In [ ]:
len(dividends)

In [ ]:
i = 0
for dividend in dividends:
    if dividend['state'] == 'paid':
        Date.append(pd.to_datetime(dividend['paid_at']))
        Transaction.append('dividend')
        Ticker.append(r.get_instrument_by_url(dividend['instrument'])['symbol'])
        Currency.append('USD')
        Price.append(float(dividend['amount'])/float(dividend['position']))
        Quantity.append(dividend['position'])
        OrderNo.append(i)
    i = i + 1 

In [ ]:
pd.to_numeric(Price)

In [ ]:
pd.to_datetime(Date)

In [ ]:
d = {'Date': pd.to_datetime(Date), 
     'Transaction': Transaction,
     'Ticker': Ticker,
     'Currency': Currency,
     'Price': pd.to_numeric(Price),
     'Quantity': pd.to_numeric(Quantity)
    }
df = pd.DataFrame(data=d)

In [ ]:
filename = '../Data/portfolio_from_py.csv'

In [ ]:
df.sort_values(by=['Date', 'Transaction'], inplace=True)

In [ ]:
df.sort_values(by=['Date', 'Transaction']).to_csv(filename, date_format='%m/%d/%y', index=False)

In [ ]:
df = df.infer_objects()

In [ ]:
df.dtypes

In [ ]:
df.loc[df.Ticker == 'CHK']

In [ ]:
df.head()

In [ ]:
df.Date = df.Date.map(lambda x: '{0}/{1:02d}/{2:02d}'.format(x.year, x.month, x.day))

In [ ]:
df.dtypes